In [45]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [46]:
product_type = "frozen"
num_time_periods = 365

test_article = '468a73f3'

In [47]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]
# article_data = article_data[(article_data['ARTICLE_ID'] == test_article)]

In [48]:
# constants definitions
if(product_type=="frozen"):
    warehouse_volume = 50
if(product_type=="chilled"):
    warehouse_volume = 300
if(product_type=="ambient"):
    warehouse_volume = 900
buffer_cost = 25
default_max_order = 10000

In [49]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [50]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [51]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [52]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [53]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start='2022-06-13', end='2022-06-18', freq='D')}).astype(str)
# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

# demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
# demand = dict((date_to_index[key],value) for (key,value) in demand.items())

demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()
for item in demand.keys():
    demand[item] = dict((date_to_index[key], value) for (key, value) in demand[item].items())

#### Heuristic

In [42]:
# demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
# demand = dict((date_to_index[key],value) for (key,value) in demand.items())

#### Method 2

In [54]:
demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()
for item in demand.keys():
    demand[item] = dict((date_to_index[key], value) for (key, value) in demand[item].items())

In [55]:
def silverMeal(item):
    
    item_demand = list(demand[item].values())
    fixed_cost = ordering_cost_fixed[item]
    variable_cost = ordering_cost_per_tu[item]
    cu_per_tu_item = cu_per_tu[item]
    disposal_cost = clearing_cost_per_cu[item]
    life = shelf_life[item]
    # item_demand = [2,3,30,5,3,10]
    # fixed_cost = 10
    # variable_cost = 1
    # cu_per_tu_item = 1
    # life = 2
    # disposal_cost = 0
    orders = {}
    # Start of the date on which we want to order 
    order_date = 0 
    run_complete = False
    obj_val = 0
    while(not run_complete):
        min_cost = float('inf')
        # On day 0 we order the demand 
        orders[order_date] = item_demand[order_date]
        # How many units can we order can we order until the cost remains lesser
        cum_demand = 0
        # Check from the start of the period until the last day 
        for t in range(order_date, len(item_demand)):
            cum_demand += item_demand[t]
            # If we reach the shelf life of an item, we order the cum_demand and start checking from the next day of the shelf life
            if(t==order_date+life):
                orders[order_date] = cum_demand
                order_date = t + 1
                break
            # Calculate the average cost which we want to minimise
            cost = fixed_cost + variable_cost * math.ceil(cum_demand/cu_per_tu_item)
            avg_cost = cost / (t-order_date+1)
            # Greedy approach where we stop as soon we find a local minima wrt to fixed, varibale cost
            if(avg_cost <= min_cost):
                min_cost = avg_cost
                # If we reach the last day of the period and we did not find a LM before that, we order all the items on that respective date
                if(t>=len(item_demand)-1):
                    orders[order_date] = cum_demand
                    cost += ((orders[order_date]*cu_per_tu_item) - cum_demand) * disposal_cost
                    run_complete = True
                    break
            # The first time avg_cost increases we jump to the next period from which we need to order 
            else:
                orders[order_date] = cum_demand - item_demand[t]
                cost += ((orders[order_date]*cu_per_tu_item) - cum_demand) * disposal_cost
                order_date = t
                break
        obj_val += cost
        # Iterate over all days of the time period
        if(order_date > len(item_demand)-1):
            break
    return (orders, obj_val)
obj_val = 0 # keep track of the objective value
for item in demand.keys():
    orders, val = silverMeal(item)  
    print(orders)
    obj_val += val
# print(silverMeal('00ee8964'))
print(obj_val)

{0: 62.0, 10: 62.0, 22: 53.0, 31: 88.0, 43: 126.0, 62: 96.0, 75: 72.0, 82: 159.0, 98: 46.0, 103: 87.0, 110: 216.0, 131: 136.0, 149: 96.0, 162: 125.0, 174: 80.0, 181: 215.0, 197: 136.0, 210: 111.0, 227: 167.0, 257: 62.0, 274: 63.0, 291: 40.0, 297: 53.0}
{0: 67.0, 7: 321.0, 19: 258.0, 32: 336.0, 54: 229.0, 72: 161.0, 86: 207.0, 103: 68.0, 113: 91.0, 127: 39.0, 136: 42.0, 145: 45.0, 159: 20.0, 167: 22.0, 176: 18.0}
{0: 171.0, 18: 223.0, 40: 110.0, 55: 188.0, 73: 127.0, 85: 127.0, 101: 79.0, 114: 128.0, 131: 79.0, 144: 94.0, 158: 93.0, 171: 78.0, 183: 77.0, 195: 77.0, 207: 123.0, 222: 95.0, 235: 107.0, 247: 124.0, 262: 111.0, 276: 76.0, 288: 111.0}
{0: 40.0, 9: 67.0, 17: 166.0, 38: 100.0, 58: 76.0, 77: 40.0, 87: 67.0, 109: 52.0, 133: 24.0}
{0: 450.0, 16: 376.0, 26: 534.0, 42: 455.0, 57: 120.0, 61: 230.0, 69: 274.0, 78: 448.0, 94: 276.0, 104: 139.0, 110: 209.0, 118: 270.0, 127: 110.0, 131: 148.0, 136: 163.0, 142: 519.0, 164: 190.0, 173: 250.0, 184: 90.0, 188: 292.0, 198: 271.0, 209: 266.0, 

#### Method 1

In [ ]:
class CostMatrixCell:
    def __init__(self) -> None:
        self.orders = {}
        self.order_date = None
        self.period_end = None
        self.cost = None
        self.avg_cost = None
        # self.extra_orders = []

class Order:
    def __init__(self) -> None:
        self.item = None
        self.quantity_tu = None
        self.date = None

# class ExtraOrder:
#     def __init__(self) -> None:
#         self.orders = {}
#         self.order_date = None
#         self.period_end = None


In [ ]:
n = 6
cost_matrix = [[CostMatrixCell() for __ in range(n)] for _ in range(n)]
for i in range(n):
    for j in range(n):
        if(i>j):
            cost_matrix[i][j] = None
        else:
            cost_matrix[i][j].order_date = i
            cost_matrix[i][j].period_end = j

def getOrders(cell):
    for t in range(cell.order_date, cell.period_end + 1):
        for item in demand[t].keys():
            if(item not in cell.orders.keys()):
                cell.orders[item] = 0
            cell.orders[item] += math.ceil(demand[t][item] / cu_per_tu[item])
            

def getCost(cell):
    cost = 0
    volume = 0
    for item in cell.orders.keys():
        life = shelf_life[item]
        cost += (ordering_cost_per_tu[item] * cell.orders[item]) + ordering_cost_fixed[item]
        volume += orders[item] * cu_per_tu[item] * volume_per_cu[item]
    buffer_cost = 0
    for t in range(cell.order_date,cell.period_end+1):
        if(volume > warehouse_volume):
            cost += (volume - warehouse_volume) * buffer_cost
            demand_volume = 0
            for item in demand[t].keys():
                demand_volume += volume_per_cu[item] * demand[t][item]
            volume -= demand_volume
        else:
            break
    
    cell.cost = cost
    cell.avg_cost = cost / (cell.period_end - cell.order_date + 1) 

for i in range(n):
    for j in range(n):
        if(type(cost_matrix[i][j])==CostMatrixCell):
            getCost(cost_matrix[i][j])

print(cost_matrix[0][0].cost)

# def getFirstMin(costs,first_index):
#     i = first_index
#     while(i < len(costs)-1 and costs[i].avg_cost > costs[i+1].avg_cost):
#         i = i+1
#     return (costs[i], i)

# obj_val = 0
# first_index = 0
# for i in range(n):
#     if(getFirstMin(cost_matrix[i], first_index)[1]+1 > n-1):
#         obj_val += getFirstMin(cost_matrix[i], first_index)[0].cost
#         break
#     obj_val += getFirstMin(cost_matrix[i], first_index)[0].cost
#     first_index = getFirstMin(cost_matrix[i], first_index)[1]+1

# obj_val

0


In [ ]:
n = 6
cost_matrix = [ [0] * n for _ in range(n)]
extra_orders_matrix = [ [None] * n for _ in range(n)]
for i in range(n):
    for j in range(n):
        if(i>j):
            cost_matrix[i][j] = float('inf')

# what and how much to order on day i to satisfy demand from day i to day j
def getOrders(i,j):
    orders = {}
    for t in range(i,j+1):
        for item in demand[t].keys():
            if(item not in orders.keys()):
                orders[item] = 0
            orders[item] += math.ceil(demand[t][item] / cu_per_tu[item])
    return orders

def getCost(i,j):
    orders = getOrders(i,j)
    cost = 0
    volume = 0 
    for item in orders.keys():
        life = shelf_life[item]
        cost += (ordering_cost_per_tu[item] * orders[item]) + ordering_cost_fixed[item]
        volume += orders[item] * cu_per_tu[item] * volume_per_cu[item]
    buffer_cost = 0
    for t in range(i,j+1):
        if(volume > warehouse_volume):
            cost += (volume - warehouse_volume) * buffer_cost
            demand_volume = 0
            for item in demand[t].keys():
                demand_volume += volume_per_cu[item] * demand[t][item]
            volume -= demand_volume
        else:
            break
    
    return cost, cost/(j-i+1) 

for i in range(n):
    for j in range(n):
        if(cost_matrix[i][j]!=float('inf')):
            cost_matrix[i][j] = getCost(i,j)

def getFirstMin(costs,first_index):
    i = first_index
    while(i < len(costs)-1 and costs[i][1] > costs[i+1][1]):
        i = i+1
    return (costs[i], i)

obj_val = 0
first_index = 0
for i in range(n):
    if(getFirstMin(cost_matrix[i], first_index)[1]+1 > n-1):
        obj_val += getFirstMin(cost_matrix[i], first_index)[0][0]
        break
    obj_val += getFirstMin(cost_matrix[i], first_index)[0][0]
    first_index = getFirstMin(cost_matrix[i], first_index)[1]+1

obj_val


10310.499999999996

In [ ]:
run_complete = False
obj_value = 0
order_date = 0

while(not run_complete):
    orders = {}
    min_cost = float('inf')
    for period_end in time_indexes[order_date:]:

        for item in demand[period_end].keys():
            if(item not in orders.keys()):
                orders[item] = 0
            orders[item] += math.ceil(demand[period_end][item] / cu_per_tu[item])

        cost = 0
        for item in orders:
            cost += (orders[item] * ordering_cost_per_tu[item]) + ordering_cost_fixed[item]
        avg_cost = cost / (period_end - order_date + 1)

        if(avg_cost <= min_cost):
            min_cost = avg_cost
            if(period_end==len(time_indexes)-1):
                obj_value += cost
                run_complete = True
        else:
            order_date = period_end
            break
obj_value

10310.499999999996

In [ ]:
run_complete = False
obj_value = 0
order_date = 0
while(not run_complete):
    ordered_items = set()
    min_cost = float('inf')
    for period_end in time_indexes[order_date:]:
        print(order_date, period_end)
        for item in demand[period_end].keys():
            if(demand[period_end][item] > 0):
                ordered_items.add(item)
        print(ordered_items)
        cost = 0
        for item in ordered_items:
            cost += ordering_cost_fixed[item]
        if(cost <= min_cost):
            min_cost = cost
            if(period_end==len(time_indexes)-1):
                run_complete = True
        else:
            order_date = period_end
            break
    obj_value += min_cost
obj_value